In [1]:
import pandas as pd
import json
import os
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
from datetime import datetime

## Actividad 1
Leer los csv y los json, además hacer las bases de datos y archivos

In [2]:
df1 = pd.read_csv('db1.csv')
df2 = pd.read_csv('db2.csv')

with open('log.json') as f:
    log_file = json.load(f)

# Carga el archivo JSON de usuarios
with open("users.json", "r") as archivo:
    users = json.load(archivo)

## Tarea 1
1. input con el nombre
2. Recorrer el diccionario trayendonos los datos a los que puede acceder cada usuario segun su nivel
3. si el usuario no existe tirar un except "no esta definido el usuario...."


In [3]:
# Función para buscar los business_name en un archivo JSON
def buscar_business_name(archivo, profile):
    with open(archivo, "r") as file:
        data = json.load(file)
        business_names = []
        for key, value in data.items():
            if "level" in value and value["level"] == profile:
                business_names.append(value["business_name"])
        return business_names

## Actividad 2
* filtrado de las primeras 5 columnas del dataset a las que el usuario tiene acceso

In [4]:
# Function to extract the node names from a JSON file based on the "profile"
def extract_node_names(archivo, profile):
    with open(archivo, "r") as file:
        data = json.load(file)
        node_names = []
        for key, value in data.items():
            if "level" in value and value["level"] == profile:
                node_names.append(key)
        return node_names

## Tarea 2
Agregar un log de registro, con usuario y timestamp

In [5]:
def save_log(user, action, data_requested=None):
    # Define the data to be added to the JSON file
    new_data = {"action": action, "timestamp": datetime.now().strftime("%d/%m/%Y %H:%M:%S")}

    # Add the "data_accessed" variable to the new data if provided
    if (action == "success_db_check" or action == "unsuccess_db_check" ) and data_requested is not None:
        new_data["data_accessed"] = data_requested

    # Specify the JSON file path
    json_file_path = "log.json"

    # Read existing data from the JSON file (if any)
    existing_data = {}
    try:
        with open(json_file_path, "r") as f:
            existing_data = json.load(f)
    except FileNotFoundError:
        # If the file doesn't exist yet, we'll handle the exception and initialize the data as an empty dictionary
        pass

    # Get the "action" and "timestamp" values from the new data
    action_value = new_data["action"]
    timestamp_value = new_data["timestamp"]

    # Check if the user exists in the existing data
    if user in existing_data:
        # Check if "activities" key exists, and initialize it if it doesn't
        if "activities" in existing_data[user]:
            existing_data[user]["activities"].append(new_data)
        else:
            existing_data[user]["activities"] = [new_data]
    else:
        # If the user doesn't exist, create a new entry with the action
        existing_data[user] = {"activities": [new_data]}


    # Save the updated data back to the JSON file
    with open(json_file_path, "w") as f:
        json.dump(existing_data, f, indent=4)

In [9]:
# Solicita al usuario que ingrese su nombre
nombre = input("Por favor, ingresa tu nombre: ")
actions = ['login','unsucessful_login','success_db_check','unsuccess_db_check']

# Verifica si el nombre de usuario está en el JSON
if nombre in users:
    # Obtiene el valor de "profile" asociado al nombre de usuario
    save_log(nombre, actions[0])
    archivo_usuarios = 'users.json'
    with open(archivo_usuarios, "r") as file:
        data = json.load(file)
    profile = data[nombre]["profile"]
    print(f"Hola {nombre}!!, tu perfil es: {profile}")
    columna_consulta = input("\n Que columna quieres consultar?")

    # Extract node names from db1.json
    node_names_db1 = extract_node_names("db1.json", profile)
    # Extract node names from db2.json
    node_names_db2 = extract_node_names("db2.json", profile)
    
    if columna_consulta in node_names_db1:
        save_log(nombre, actions[2],columna_consulta)
        print(f'Claro {nombre}, aqui tienes la información de {columna_consulta} \n\n')
        print(df1.loc[:, columna_consulta].head())
    elif columna_consulta in node_names_db2:
        save_log(nombre, actions[2],columna_consulta)
        print(f'Claro {nombre}, aqui tienes la información de {columna_consulta} \n\n')
        print(df2.loc[:, columna_consulta].head())
    else:
        save_log(nombre, actions[3],columna_consulta)
        print(f'Ese dato {columna_consulta} no existe en la base de datos')

else:
    save_log(nombre, actions[1])
    print(f"Lo siento {nombre} , no se encontró tu nombre en la base de Usuarios.")

Hola Carlos!!, tu perfil es: PI
Ese dato Location no existe en la base de datos


In [10]:

def save_log(user):
    # Define the data to be added to the JSON file
    new_data = {"date_hour": datetime.now().strftime("%d/%m/%Y %H:%M:%S"), "user": user}

    # Specify the JSON file path
    json_file_path = "log.json"

    # Read existing data from the JSON file (if any)
    existing_data = {}
    try:
        with open(json_file_path, "r") as f:
            existing_data = json.load(f)
    except FileNotFoundError:
        # If the file doesn't exist yet, we'll handle the exception and initialize the data as an empty dictionary
        pass

    # Get the "user" value from the new data
    user_value = new_data["user"]

    # Append the new "date_hour" to the list associated with the "text" value
    if user_value in existing_data:
        existing_data[user_value]["date_hour"].append(new_data["date_hour"])
    else:
        existing_data[user_value] = {"date_hour": [new_data["date_hour"]]}

    # Save the updated data back to the JSON file
    with open(json_file_path, "w") as f:
        json.dump(existing_data, f, indent=4)
